ANTIKYTHERA PLAYGROUND DATATHON 2024

In [ ]:
pip install numpy
pip install matplotlib
pip install pandas

In [4]:
import numpy as np
import pandas as pd

DataTest = pd.read_csv("test.csv")
DataIp = pd.read_csv("ip_address_mapping.csv")

In [ ]:
DataTest

In [ ]:
DataIp

In [11]:
Data = pd.concat([DataIp, DataTest], axis=1)